# Linear regression

In [1]:
import numpy as np
import pandas as pd
import sklearn
import math
from utils import DataHandlerTitantic

Load the data and split into train and validation sets.

In [2]:
dh = DataHandlerTitantic(34545234)
dh.load_train_data("data/train.csv")
dh.load_test_data("data/test.csv")
dh.shuffle_split()
dh.data_full_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Gender and socio-economic status

Convert the 'Sex' column into an integer one, and replace 'Pclass' with a categorical one.

In [3]:
dh.to_is_female()
dh.make_dummies(["Pclass"])
dh.train.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_1,Pclass_2,Pclass_3
763,764,1,"Carter, Mrs. William Ernest (Lucile Polk)",1,36.0,1,2,113760,120.000,B96 B98,S,1,0,0
632,633,1,"Stahelin-Maeglin, Dr. Max",0,32.0,0,0,13214,30.500,B50,C,1,0,0
237,238,1,"Collyer, Miss. Marjorie ""Lottie""",1,8.0,0,2,C.A. 31921,26.250,NaN,S,0,1,0
491,492,0,"Windelov, Mr. Einar",0,21.0,0,0,SOTON/OQ 3101317,7.250,NaN,S,0,0,1
590,591,0,"Rintamaki, Mr. Matti",0,35.0,0,0,STON/O 2. 3101273,7.125,NaN,S,0,0,1
